# Demonstrate asyncio environment

The mango framework needs to run in an asyncio loop

 - loop run as long as tasks are available (start with asyncio.run)
 - tasks can be dispatched to the loop (asyncio.create_task)
 - the loop/scheduler inside the loop determines when which task should be executed (control delays with asyncio.sleep)
 - special keyworkds async, await

Careful! The behavior when using .py files directly is different. In Jupyternotebooks there is always an asyncio loop running

In [1]:
import asyncio
# coroutine function
async def main():
    print("From an async function")

coroutine = main()
# it be 



There are two entities you should know about.

First, there are agents in mango, which represen the main acting unit in mango (same concept as in the lecture).

In [2]:
import mango

class SimpleAgent(mango.Agent):
    pass

There are Container. Container contain Agents, and build the backbone for the communication capabilities, which enable agent-based cooperation

Container also abstract the communication technologie (in mango TCP and MQTT exist) and the codec (JSON, protobuf -> format in which messages are serialized)

Serialization is necessary as there need to be a well-defined format which can be interpreted on different machines.

In [3]:
import mango

container = mango.create_tcp_container("127.0.0.1:9999")

# you need to register agents to the container
agent = container.register(SimpleAgent())

# Demonstrate agent implementation


In [4]:
import mango

class MyAgent(mango.Agent):
    # init
    # on_register
    # on_ready
    # on_start

    def __init__(self, name=None):
        super().__init__()
        self.name = name
        print(f"Creating a RepeatingAgent. At this point self.addr={self.addr}")

    def handle_message(self, content, meta):
        # This method defines what the agent will do with incoming messages.
        print(f"Received a message with the following content: {content} {meta}!")

    def on_register(self):
        print(f"The agent has been registered to a container: {self.addr}!")

    def on_ready(self):
        print("All containers have been activated!")


container = mango.create_tcp_container("127.0.0.1:9999")
agent = container.register(MyAgent())

async with mango.activate(container):
    pass


Creating a RepeatingAgent. At this point self.addr=None
The agent has been registered to a container: AgentAddress(protocol_addr=('127.0.0.1', 9999), aid='agent0')!
All containers have been activated!


# Demonstrate container activation


In [5]:
import mango

container = mango.create_tcp_container("127.0.0.1:9999")

# async with mango.activate(container) as c:
#     pass

print(container)

# Demonstrate message sending

In [6]:
import mango 

container = mango.create_tcp_container("127.0.0.1:9999")
agent = container.register(mango.PrintingAgent())

async with mango.activate(container) as c:
    pass
    # await container.send_message("Hi there!", agent.addr)


# Demonstrate message handling

In [10]:
import mango 

class MyPrintingAgent(mango.Agent):

    def handle_message(self, content, meta):
        print(f"Received message: {content} with meta: {meta}")

container = mango.create_tcp_container("127.0.0.1:9999")
agent = container.register(mango.PrintingAgent())

async with mango.activate(container) as c:
    await container.send_message("Hi there!", agent.addr)


Received: Hi there! with {'sender_id': None, 'sender_addr': ('127.0.0.1', 9999), 'receiver_id': 'agent0', 'network_protocol': 'tcp', 'priority': 0}
